everything is already in a tablular form on this pdf so super simple to get it into a dataframe

In [2]:
import pandas as pd
import tabula

# Specify the path to the PDF file
pdf_path = 'pdfs/food.pdf'


tables = tabula.read_pdf(pdf_path, pages='all', multiple_tables=True)

# Concatenate all tables into a single dataframe
df = pd.concat(tables)

# Print the dataframe
print(df)

Failed to import jpype dependencies. Fallback to subprocess.
No module named 'jpype'


       ID TYPE                                            PROGRAM  \
0   85670   FP  ACADEMY OF MEDICAL & PUBLIC HEALTH SERVICES (A...   
1   84948   FP             ACTS COMMUNITY DEVELOPMENT CORPORATION   
2   84664   SK             ACTS COMMUNITY DEVELOPMENT CORPORATION   
3   85247   FP                               ADMIRABLE CHURCH INC   
4   85798   FP                              AL-MADINAH SCHOOL INC   
..    ...  ...                                                ...   
20  82139   FP                          STAPLETON U.A.M.E. CHURCH   
21  81360   SK                          STAPLETON U.A.M.E. CHURCH   
22  81039   FP                 THE SALVATION ARMY STAPLETON CORPS   
23  81040   SK                 THE SALVATION ARMY STAPLETON CORPS   
24  85326   FP  UNITED CHURCH OF PRAISE INTERNATIONAL MINISTRI...   

         ORG PHONE                 DISTADDDI  TB DRIOSTZIP       DAYS  \
0     212-256-9036  5306 3RD AVENUE - 2ND FL  BK     11220   SAT(1,3)   
1   (718) 415-1170       

In [3]:
df['FULL_ADDRESS'] = df['DISTADDDI'].astype(str) + ', ' + df['TB'].astype(str) + ', ' + df['DRIOSTZIP'].astype(str)
df

,ID,TYPE,PROGRAM,ORG PHONE,DISTADDDI,TB,DRIOSTZIP,DAYS,HOURS,FULL_ADDRESS
0,85670,FP,ACADEMY OF MEDICAL & PUBLIC HEALTH SERVICES (A...,212-256-9036,5306 3RD AVENUE - 2ND FL,BK,11220,"SAT(1,3)",2-4PM,"5306 3RD AVENUE - 2ND FL, BK, 11220"
1,84948,FP,ACTS COMMUNITY DEVELOPMENT CORPORATION,(718) 415-1170,2114 MERMAID AVE,BK,11224,TUE/THUR,6-7:15M/11AM-2:15PM,"2114 MERMAID AVE, BK, 11224"
2,84664,SK,ACTS COMMUNITY DEVELOPMENT CORPORATION,718-415-1170,2114 MERMAID AVE,BK,11224,WED/FRI,6-7:15PM,"2114 MERMAID AVE, BK, 11224"
3,85247,FP,ADMIRABLE CHURCH INC,347-779-7192,1910 FLATBUSH AVENUE,BK,11210,SAT (4),10AM-12PM,"1910 FLATBUSH AVENUE, BK, 11210"
4,85798,FP,AL-MADINAH SCHOOL INC,718-222-4986,383 3RD AVENUE,BK,11215,FRI (3RD),12:30-2:30PM,"383 3RD AVENUE, BK, 11215"
...,...,...,...,...,...,...,...,...,...,...
20,82139,FP,STAPLETON U.A.M.E. CHURCH,(718) 273-2857,49 TOMPKINS AVE,SI,10304,"TUE,FRI",10AM-1PM,"49 TOMPKINS AVE, SI, 10304"
21,81360,SK,STAPLETON U.A.M.E. CHURCH,(718) 273-2857,49 TOMPKINS AVENUE,SI,10304,MON/WED,3-4PM/12-1PM,"49 TOMPKINS AVENUE, SI, 10304"
22,81039,FP,THE SALVATION ARMY STAPLETON CORPS,(718) 448-8480,15 BROAD STREET,SI,10304,TUE-THUR,12-1:30PM,"15 BROAD STREET, SI, 10304"
23,81040,SK,THE SALVATION ARMY STAPLETON CORPS,(718) 448-8480,15 BROAD STREET,SI,10304,"MON,FRI",11:30AM-1PM,"15 BROAD STREET, SI, 10304"


In [4]:
from geopy.geocoders import Nominatim
from geopy.exc import GeocoderTimedOut
from tqdm import tqdm
import time
import pandas as pd

# Initialize geocoder
geolocator = Nominatim(user_agent="my_app")

# Combine address components into a single address string
df['FULL_ADDRESS'] = df['DISTADDDI'].astype(str) + ', ' + df['TB'].astype(str) + ', ' + df['DRIOSTZIP'].astype(str)
df['LONGITUDE'] = None
df['LATITUDE'] = None

# Function to get coordinates with retry logic
def get_coordinates(address, geolocator, retries=3):
    for _ in range(retries):
        try:
            location = geolocator.geocode(address, timeout=10)
            if location:
                return location.latitude, location.longitude
            else:
                return None, None
        except GeocoderTimedOut:
            time.sleep(1)
    return None, None

# Apply geocoding with progress bar
for idx, address in tqdm(enumerate(df['FULL_ADDRESS']), total=len(df), desc="Geocoding Addresses"):
    latitude, longitude = get_coordinates(address, geolocator)
    df.at[idx, 'LATITUDE'] = latitude
    df.at[idx, 'LONGITUDE'] = longitude

# Print the updated DataFrame
print(df)


Geocoding Addresses: 100%|██████████| 557/557 [06:46<00:00,  1.37it/s]


          ID TYPE                                            PROGRAM  \
0    85670.0   FP  ACADEMY OF MEDICAL & PUBLIC HEALTH SERVICES (A...   
1    84948.0   FP             ACTS COMMUNITY DEVELOPMENT CORPORATION   
2    84664.0   SK             ACTS COMMUNITY DEVELOPMENT CORPORATION   
3    85247.0   FP                               ADMIRABLE CHURCH INC   
4    85798.0   FP                              AL-MADINAH SCHOOL INC   
..       ...  ...                                                ...   
552      NaN  NaN                                                NaN   
553      NaN  NaN                                                NaN   
554      NaN  NaN                                                NaN   
555      NaN  NaN                                                NaN   
556      NaN  NaN                                                NaN   

          ORG PHONE                 DISTADDDI   TB DRIOSTZIP       DAYS  \
0      212-256-9036  5306 3RD AVENUE - 2ND FL   BK     11220

In [5]:
df.dropna(subset=['TYPE'], inplace=True)

In [6]:
df.head()

,ID,TYPE,PROGRAM,ORG PHONE,DISTADDDI,TB,DRIOSTZIP,DAYS,HOURS,FULL_ADDRESS,LONGITUDE,LATITUDE
0,85670.0,FP,ACADEMY OF MEDICAL & PUBLIC HEALTH SERVICES (A...,212-256-9036,5306 3RD AVENUE - 2ND FL,BK,11220,"SAT(1,3)",2-4PM,"5306 3RD AVENUE - 2ND FL, BK, 11220",None,None
1,84948.0,FP,ACTS COMMUNITY DEVELOPMENT CORPORATION,(718) 415-1170,2114 MERMAID AVE,BK,11224,TUE/THUR,6-7:15M/11AM-2:15PM,"2114 MERMAID AVE, BK, 11224",-73.988415,40.576042
2,84664.0,SK,ACTS COMMUNITY DEVELOPMENT CORPORATION,718-415-1170,2114 MERMAID AVE,BK,11224,WED/FRI,6-7:15PM,"2114 MERMAID AVE, BK, 11224",-73.988415,40.576042
3,85247.0,FP,ADMIRABLE CHURCH INC,347-779-7192,1910 FLATBUSH AVENUE,BK,11210,SAT (4),10AM-12PM,"1910 FLATBUSH AVENUE, BK, 11210",-73.937721,40.623107
4,85798.0,FP,AL-MADINAH SCHOOL INC,718-222-4986,383 3RD AVENUE,BK,11215,FRI (3RD),12:30-2:30PM,"383 3RD AVENUE, BK, 11215",-73.988272,40.674283


In [7]:
null_longitude = df['LONGITUDE'].isnull().sum()
null_latitude = df['LATITUDE'].isnull().sum()

print("Number of null values in LONGITUDE column:", null_longitude)
print("Number of null values in LATITUDE column:", null_latitude)

Number of null values in LONGITUDE column: 85
Number of null values in LATITUDE column: 85
